## The simplest workflow pattern executes steps in a predefined order. Each step's output becomes input for the next step, creating a clear chain of operations. This pattern is ideal for tasks with well-defined sequences, like content generation pipelines or data transformation processes.

In [2]:
import { generateText, generateObject } from 'ai'
import { z } from 'zod'
import { createCompatibleOpenAI } from '../../init.ts'

const openai = createCompatibleOpenAI()

const qualityMetricsSchema = z.object({
	persuasion: z.boolean(),
	clarity: z.number(),
	creativity: z.number(),
})


async function generateMarketingCopy(input: string) {
	const model = openai('gpt-4o-mini')

	// 使用模型生成营销文案
	const { text: copy } = await generateText({
		model,
		prompt:`
		You are a marketing copywriter.
		You are given a product description and you need to generate a persuasive marketing copy for the product.
		
		Product description: ${input}	
		`,
	})


	// 生成文案质量评估
	const { object: qualityMetrics } = await generateObject<z.infer<typeof qualityMetricsSchema>>({
		model,
		schema: qualityMetricsSchema,
		prompt: `
		Evaluate this marketing copy for:
		- Persuasion: How effectively does the copy persuade the reader to take action(true or false)?
		- Clarity: Is the copy clear and easy to understand(1-10)?
		- Creativity: Is the copy creative and engaging(1-10)?
		
		Marketing copy: ${copy}
		`
	})
	
	
	// 如果文案质量不通过，则重新生成文案
	if (!qualityMetrics.persuasion || qualityMetrics.clarity < 8 || qualityMetrics.creativity < 8) {
		const { text: improvedCopy } = await generateText({
			model,
			prompt: `
			Improve this marketing copy for:
			- ${qualityMetrics.persuasion ? '' : 'Persuasion is not effective'}
			- ${qualityMetrics.clarity < 8 ? `Improve the clarity and make it easier to understand` : ''}
			- ${qualityMetrics.creativity < 8 ? `Make it more creative and engaging` : ''}
				
				Marketing copy: ${copy}
			`
		})

		return {
			copy: improvedCopy,
			qualityMetrics
		}
	}

	return { copy, qualityMetrics }
}


const { copy, qualityMetrics } = await generateMarketingCopy('A product that helps people learn to code')
console.log(copy)
console.log(qualityMetrics)

TypeError: Relative import path "init" not prefixed with / or ./ or ../ and not in import map from "file:///Users/luyao/learn-front-end/learn-frontend-ai/learn-vercel-ai-sdk/foundations/agents/$deno$repl.mts"